In [2]:
import torch
import numpy as np
import torch.nn as nn
import sys
import os
project_root = os.path.abspath("../..")  # Adjust if needed
import pytorch_lightning as pl
# Add the project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

# from src.models.pointNetVae import PointNetVAE

from src.models.PointNetVae_chamfer_split import PointNetVAE
from src.utils.data_utils import *
from src.dataset_classes.pointDataset import *
from proteinshake.datasets import ProteinFamilyDataset
from proteinshake.tasks import LigandAffinityTask
import random
from torch.utils.data import DataLoader, Dataset, Subset, TensorDataset
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
%load_ext autoreload
%autoreload 2

In [4]:
dataset = ProteinFamilyDataset(root='../../data').to_point().torch()
idx_list = range(len(dataset))
subset_size = int(len(dataset)//10)
val_idx = random.sample(idx_list, subset_size)  # Get random subset
train_idx = list(set(idx_list) - set(val_idx))

test_subset =  PointDataset(Subset(dataset, train_idx), 500, return_proteins=True)
filter_output = filter_by_max_length_and_pad(dataset,500, True)

train_max = find_max_radius(test_subset)

100%|██████████| 25875/25875 [00:00<00:00, 45915.67it/s]


In [ ]:
def calculate_enhanced_chamfer_distance(self, predicted_coords, true_coords, mask):
    # Get L2 distances across points
    AB_pwd = torch.cdist(predicted_coords, x2=true_coords, p=2)

    # Create mask
    mask = (mask.unsqueeze(-1).float() @ mask.unsqueeze(1).float()).int().bool()
    masked_dist_matrix = torch.where(mask, AB_pwd, torch.tensor(float('inf'), device=mask.device)) 

    # Get min distance from A to B and B to A and sum
    min_predicted_to_true, pred_to_true_min_idx = torch.min(masked_dist_matrix, dim = 1)
    min_true_to_pred, min_true_to_pred_idx = torch.min(masked_dist_matrix, dim = -1)
    
    non_inf_sum = torch.sum(torch.where(torch.min(masked_dist_matrix, dim=-1)[0] != torch.inf, 
                                        torch.min(masked_dist_matrix, dim=-1)[0], 
                                        torch.tensor(0)), dim = -1)
    

    return 2*torch.mean(non_inf_sum)
test_subset[0]

torch.Size([500, 24])

In [ ]:
x = test_subset[0]
x_true_indices = x[:,3:]
x_true_indices = x_true_indices.argmax(dim=-1)
x_true_indices[torch.where(torch.sum(x[:,3:], dim = -1) == 0)] = -1

mask = (x_true_indices != -1)

AB_pwd = torch.cdist(test_subset[1][:], x2=x, p=2)

In [26]:
test_subset[1][0]

tensor([ 0.0694, -0.0862, -0.0261,  0.0000,  0.0000,  0.0000,  1.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000])

In [30]:
torch.sqrt(torch.sum((x[239] - test_subset[1][0])**2))

tensor(0.0859)

In [31]:
torch.min(AB_pwd, dim = -1)[-1]

tensor([239,  34,  32,  28, 247,  97, 213,  42,  57, 252,  59,  38,  55, 256,
        209, 212, 212, 186, 216, 259, 177, 124, 151, 213, 213, 154, 213, 144,
        144, 100, 192, 144, 166, 171,  83,  83, 118,  95, 204, 118, 205, 204,
        203, 191, 189, 151, 194, 191, 222, 207,  87, 206, 197, 189, 191, 151,
        190, 225, 178, 217, 217, 217, 219, 175, 209, 261, 215, 254,  83, 209,
        254,  58,  83,  59,  83, 252, 254,  85, 249, 248,  62,  34, 238, 254,
        242, 246, 247,  94, 246,  91,  88,  90, 118, 112, 112, 118, 118, 108,
        112,  82,  67, 104, 104, 127, 127,  99,  97, 217, 213, 124, 186, 127,
        169, 189, 217, 177, 190, 183, 150,  83, 274, 277, 278, 278, 278, 278,
        278, 278, 278, 278, 278, 278, 278, 278, 278, 278, 278, 278, 278, 278,
        278, 278, 278, 278, 278, 278, 278, 278, 278, 278, 278, 278, 278, 278,
        278, 278, 278, 278, 278, 278, 278, 278, 278, 278, 278, 278, 278, 278,
        278, 278, 278, 278, 278, 278, 278, 278, 278, 278, 278, 2

In [ ]:
AB_pwd

torch.Size([500, 500])